In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
dbname = 'cdc'
username = 'Drew'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))
con = psycopg2.connect(database = dbname, user = username)

### (All data available in .xpt format from https://wwwn.cdc.gov/Nchs/Nhanes/ContinuousNhanes/Default.aspx)
### At the time of this project, datasets from 1999 through 2013 were available, binned into ~10,000 samples per every 2 years
##### Note that many questions from the survey offer a 'don't know' or 'refused to answer' catogory.  Unfortunately the coding of these responses varies from question to question (depending on range, they take on values of 7, 77, 777... up to 777777, and likewise with 9, 99, 999...  As data is cleaned and combined, these values are recoded to numpy.nans, to be dealt with via dropping/imputation once we get to the modeling section.

# TARGETS: 

## Blood Pressure

In [2]:
# import and combine data sets from 1999-2013
bpx99 = pd.read_csv('CDCfiles/BPX.csv')
bpx01 = pd.read_csv('CDCfiles/BPX_B.csv')
bpx03 = pd.read_csv('CDCfiles/BPX_C.csv')
bpx05 = pd.read_csv('CDCfiles/BPX_D.csv')
bpx07 = pd.read_csv('CDCfiles/BPX_E.csv')
bpx09 = pd.read_csv('CDCfiles/BPX_F.csv')
bpx11 = pd.read_csv('CDCfiles/BPX_G.csv')
bpx13 = pd.read_csv('CDCfiles/BPX_H.csv')

In [3]:
# join together years 1999-2013
bpx = bpx99.append(bpx01).append(bpx03).append(bpx05).append(bpx07).append(bpx09).append(bpx11).append(bpx13)

# recode blood pressure (systolic and diastolic) as average of 2 separate measurements
bpx['BPXsys'] = bpx[['BPXSY1', 'BPXSY2']].mean(axis=1)
bpx['BPXdi'] = bpx[['BPXDI1', 'BPXDI2']].mean(axis=1)

In [4]:
# pull out just the measurements we want
bpxF = bpx[['SEQN', 'BPXsys', 'BPXdi']]

## Cholesterol

In [5]:
# import all years of datasets containing measurements of HDL, LDL, total cholesterol, and triglycerides
chol99HD = pd.read_csv('CDCfiles/LAB13.csv')
chol01HD = pd.read_csv('CDCfiles/L13_B.csv')
chol03HD = pd.read_csv('CDCfiles/L13_C.csv')
chol05HD = pd.read_csv('CDCfiles/HDL_D.csv')
chol07HD = pd.read_csv('CDCfiles/HDL_E.csv')
chol09HD = pd.read_csv('CDCfiles/HDL_F.csv')
chol11HD = pd.read_csv('CDCfiles/HDL_G.csv')
chol13HD = pd.read_csv('CDCfiles/HDL_H.csv')

chol99LD = pd.read_csv('CDCfiles/LAB13AM.csv')
chol01LD = pd.read_csv('CDCfiles/L13AM_B.csv')
chol03LD = pd.read_csv('CDCfiles/L13AM_C.csv')
chol05LD = pd.read_csv('CDCfiles/TRIGLY_D.csv')
chol07LD = pd.read_csv('CDCfiles/TRIGLY_E.csv')
chol09LD = pd.read_csv('CDCfiles/TRIGLY_F.csv')
chol11LD = pd.read_csv('CDCfiles/TRIGLY_G.csv')
chol13LD = pd.read_csv('CDCfiles/TRIGLY_H.csv')

chol05tot = pd.read_csv('CDCfiles/TCHOL_D.csv')
chol07tot = pd.read_csv('CDCfiles/TCHOL_E.csv')
chol09tot = pd.read_csv('CDCfiles/TCHOL_F.csv')
chol11tot = pd.read_csv('CDCfiles/TCHOL_G.csv')
chol13tot = pd.read_csv('CDCfiles/TCHOL_H.csv')

# join together separate measurements into a single file for each year
chol99 = chol99HD.set_index('SEQN').join(chol99LD.set_index('SEQN'))
chol01 = chol01HD.set_index('SEQN').join(chol01LD.set_index('SEQN'))
chol03 = chol03HD.set_index('SEQN').join(chol03LD.set_index('SEQN'))
chol05 = chol05tot.set_index('SEQN').join(chol05HD.set_index('SEQN')).join(chol05LD.set_index('SEQN'))
chol07 = chol07tot.set_index('SEQN').join(chol07HD.set_index('SEQN')).join(chol07LD.set_index('SEQN'))
chol09 = chol09tot.set_index('SEQN').join(chol09HD.set_index('SEQN')).join(chol09LD.set_index('SEQN'))
chol11 = chol11tot.set_index('SEQN').join(chol11HD.set_index('SEQN')).join(chol11LD.set_index('SEQN'))
chol13 = chol13tot.set_index('SEQN').join(chol13HD.set_index('SEQN')).join(chol13LD.set_index('SEQN'))

#recode measurements from older studies to match same measurements from more recent studies
chol99['LBDHDD'] = chol99['LBDHDL']
chol01['LBDHDD'] = chol01['LBDHDL']

In [6]:
# join together years 1999-2013
chol = chol99.append(chol01).append(chol03).append(chol05).append(chol07).append(chol09).append(chol11).append(chol13)
chol['SEQN'] = chol.index

In [7]:
# pull out just the measurements we want
cholF = chol[['SEQN', 'LBXTC', 'LBDHDD', 'LBDLDL', 'LBXTR']]

# Diabetes

In [8]:
# import all years of datasets
ghb99 = pd.read_csv('CDCfiles/GHB.csv')
ghb01 = pd.read_csv('CDCfiles/GHB_B.csv')
ghb03 = pd.read_csv('CDCfiles/GHB_C.csv')
ghb05 = pd.read_csv('CDCfiles/GHB_D.csv')
ghb07 = pd.read_csv('CDCfiles/GHB_E.csv')
ghb09 = pd.read_csv('CDCfiles/GHB_F.csv')
ghb11 = pd.read_csv('CDCfiles/GHB_G.csv')
ghb13 = pd.read_csv('CDCfiles/GHB_H.csv')

In [9]:
#join together years 1999-2013
ghb = ghb99.append(ghb01).append(ghb03).append(ghb05).append(ghb07).append(ghb09).append(ghb11).append(ghb13)

In [10]:
# pull out just the measurements we want
ghbF = ghb[['SEQN', 'LBXGH']]

# FEATURES:

## Demographics - 82,091 entires total

'RIAGENDR' =  1-male  // 2-female  
'RIDAGEYR' =  0-85 numerical  
'RIDRETH1 =  categorical ethnicity, for get_dummies  
'DMDBORN4' =  1-US // 2-elsewhere    
'DMDEDUC2' =  1: less then high school through //  5:college or higher    ......NANs: 50%      
'DMDMARTL' =  1: married // 2: divorced/widowed  // 3: never married   ......NANs: 50%  
'INDHHIN2' =  income binned as 1: 20k // 2: 20-75k // 3: 75k  ......NANs: 7%    
'DMDHHSIZ' =  household size 0-7 numerical  
'INDFMPIR' =  ratio of income/poverty line      ......NANs 10%  

In [11]:
#import all years of demographics info
demo99 = pd.read_csv('CDCfiles/DEMO.csv')
demo01 = pd.read_csv('CDCfiles/DEMO_B.csv')
demo03 = pd.read_csv('CDCfiles/DEMO_C.csv')
demo05 = pd.read_csv('CDCfiles/DEMO_D.csv')
demo07 = pd.read_csv('CDCfiles/DEMO_E.csv')
demo09 = pd.read_csv('CDCfiles/DEMO_F.csv')
demo11 = pd.read_csv('CDCfiles/DEMO_G.csv')
demo13 = pd.read_csv('CDCfiles/DEMO_H.csv')

In [12]:
# combine years 1999 through 2005, which have common codes
demoold = demo99.append(demo01).append(demo03).append(demo05)

# recode country of birth to the name used in later years, convert to 2 class problem (born in US vs not in US)
demoold['DMDBORN4'] = demoold['DMDBORN']
demoold['DMDBORN4'][demoold['DMDBORN'] == 3.0] = 2.0

# recode income into 3 categories
demoold['INDHHIN2'] = demoold['INDHHINC']
demoold['INDHHIN2'][demoold['INDHHIN2'] < 5.0] = 1.0
demoold['INDHHIN2'][demoold['INDHHIN2'] == 13.0] = 1.0
demoold['INDHHIN2'][demoold['INDHHIN2'] == 11.0] = 3.0
demoold['INDHHIN2'][demoold['INDHHIN2'] > 75.0] = np.nan
demoold['INDHHIN2'][demoold['INDHHIN2'] > 4.0] = 2.0

# recode elements as they change in 2007, 2009
demo07['DMDBORN4'] = demo07['DMDBORN2']
demo07['DMDBORN4'][demo07['DMDBORN4'] == 4.0] = 2.0
demo07['DMDBORN4'][demo07['DMDBORN4'] == 5.0] = 2.0

demo09['DMDBORN4'] = demo09['DMDBORN2']
demo09['DMDBORN4'][demo09['DMDBORN4'] == 4.0] = 2.0
demo09['DMDBORN4'][demo09['DMDBORN4'] == 5.0] = 2.0

#recode race in later years back into the RIDRETH1 variabel, but maintain tje new 'asian' class
demo11['RIDRETH1'][demo11['RIDRETH3'] == 6.0] = 6.0
demo13['RIDRETH1'][demo13['RIDRETH3'] == 6.0] = 6.0
demo11['RIDRETH1'][demo11['RIDRETH3'] == 7.0] = 5.0
demo13['RIDRETH1'][demo13['RIDRETH3'] == 7.0] = 5.0

#combine 2007 through 2013
demonew = demo07.append(demo09).append(demo11).append(demo13)

# recode income into the bins described above
demonew['INDHHIN2'][demonew['INDHHIN2'] < 5.0] = 1.0
demonew['INDHHIN2'][demonew['INDHHIN2'] == 13.0] = 1.0
demonew['INDHHIN2'][demonew['INDHHIN2'] == 14.0] = 3.0
demonew['INDHHIN2'][demonew['INDHHIN2'] == 15.0] = 3.0
demonew['INDHHIN2'][demonew['INDHHIN2'] > 75.0] = np.nan
demonew['INDHHIN2'][demonew['INDHHIN2'] > 4.0] = 2.0

In [13]:
# combine demoold (1999-2005) with demonew (2007-2013)
demo = demoold.append(demonew)

# recode gender so 0 = female, 1 = male
demo['RIAGENDR'][demo['RIAGENDR'] == 2.0] = 0.0

# recode age so that years including subjects > 80 are recoded to 80
demo['RIDAGEYR'][demo['RIDAGEYR'] > 80.0] = 80.0

# recode missing values
demo['DMDBORN4'][demo['DMDBORN4'] > 3.0] = np.nan
demo['DMDEDUC2'][demo['DMDEDUC2'] > 6.0] = np.nan

# convert marital stats answers to 3 categories described above
demo['DMDMARTL'][demo['DMDMARTL'] == 6.0] = 1.0
demo['DMDMARTL'][demo['DMDMARTL'] == 3.0] = 2.0
demo['DMDMARTL'][demo['DMDMARTL'] == 4.0] = 2.0
demo['DMDMARTL'][demo['DMDMARTL'] == 5.0] = 3.0
demo['DMDMARTL'][demo['DMDMARTL'] > 70.0] = np.nan

In [14]:
# pull out just the measurements we want
demoF = demo[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'DMDBORN4', 'DMDEDUC2', 'INDHHIN2', 'DMDHHSIZ', 'DMDMARTL', 'INDFMPIR']]

In [15]:
# convert racial background to one-hot encoding
demoFhot = (pd.get_dummies(demo.RIDRETH1))
demoFdummy = pd.concat([demoF, demoFhot], axis=1)

# Body Type - 68,705 entries

'BMXHT' is missing 10%
'BMXWT' is missing 2%  
'BMXBMI' is missing 10%   
'BMXWAIST' is missing 13%   

In [16]:
# import data from 1999-2013
body99 = pd.read_csv('CDCfiles/BMX.csv')
body01 = pd.read_csv('CDCfiles/BMX_B.csv')
body03 = pd.read_csv('CDCfiles/BMX_C.csv')
body05 = pd.read_csv('CDCfiles/BMX_D.csv')
body07 = pd.read_csv('CDCfiles/BMX_E.csv')
body09 = pd.read_csv('CDCfiles/BMX_F.csv')
body11 = pd.read_csv('CDCfiles/BMX_G.csv')
body13 = pd.read_csv('CDCfiles/BMX_H.csv')

In [17]:
# combine data from 1999-2013
body = body99.append(body01).append(body03).append(body05).append(body07).append(body09).append(body11)

In [18]:
# pull out just the features we want
bodyF = body[['SEQN', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXWAIST']]

## Alcohol Use - 42,291 entries

'ALQ120U/Q' = how often you drank in the last 12 months     .........NANs: 25%  
'ALQ130' ...= avg # of drinks on those occasions when you drank  ......NANs: 25%  
'ALQpoly' ..= 120x130 = how much you drank in the last year  ......NANs: 25%   
'ALQ141U/Q' = how often you had >4 drinks in last 12 months  ......NANs: 40% (mostly 0's though)    
'ALQ151' ...= yes/no ever had a phase of >4 drinks every day  ...... NANs: 25%


__try dropping NANs from ALQ120, then imputing those that remain (keep the ALQ141 NANs)__

In [19]:
# import all datasets from 1999-2013
alc99 = pd.read_csv('CDCfiles/ALQ.csv')
alc01 = pd.read_csv('CDCfiles/ALQ_B.csv')
alc03 = pd.read_csv('CDCfiles/ALQ_C.csv')
alc05 = pd.read_csv('CDCfiles/ALQ_D.csv')
alc07 = pd.read_csv('CDCfiles/ALQ_E.csv')
alc09 = pd.read_csv('CDCfiles/ALQ_F.csv')
alc11 = pd.read_csv('CDCfiles/ALQ_G.csv')
alc13 = pd.read_csv('CDCfiles/ALQ_H.csv')

In [20]:
# combine data from 1999-2009
alcold = alc99.append(alc01).append(alc03).append(alc05).append(alc07).append(alc09)

In [21]:
# recode to match the coding post 2009
alcold['ALQ141Q'] = alcold['ALQ140Q']
alcold['ALQ141U'] = alcold['ALQ140U']
alcold['ALQ151'] = alcold['ALQ150']

In [22]:
# combine alcold with data from 2011 and 2013 (with the same coding)
alc = alcold.append(alc11).append(alc13)

In [23]:
# convert NaNs in unit column (and 130) to 0.0 where quantity = 0.0 (to preserve 0s after multiplication step)
alc['ALQ120U'][alc['ALQ120Q'] == 0.0] = 0.0
alc['ALQ130'][alc['ALQ120Q'] == 0.0] = 0.0
alc['ALQ141U'][alc['ALQ141Q'] == 0.0] = 0.0
#recode NaNs
alc['ALQ120Q'][alc['ALQ120Q'] > 775.0] = np.nan
alc['ALQ141Q'][alc['ALQ141Q'] > 775.0] = np.nan
alc['ALQ130'][alc['ALQ130'] > 750] = np.nan
alc['ALQ151'][alc['ALQ151'] > 6.0] = np.nan

# fix the scaling for the unit column, then multiply with the quantity column
alc['ALQ120U'] = alc['ALQ120U'].replace([0.0, 1.0, 2.0, 3.0, 7.0, 9.0], [0.0, 52.0, 12.0, 1.0, np.nan, np.nan])
alc['ALQ141U'] = alc['ALQ141U'].replace([0.0, 1.0, 2.0, 3.0, 7.0, 9.0], [0.0, 52.0, 12.0, 1.0, np.nan, np.nan])
alc['ALQ120'] = alc['ALQ120Q']*alc['ALQ120U']
alc['ALQ141'] = alc['ALQ141Q']*alc['ALQ141U']

# create ALQpoly by multiplying frequency of drinking by volume
alc['ALQpoly'] = alc['ALQ120']*alc['ALQ130']

In [24]:
# pull out just the features we want
alcF = alc[['SEQN', 'ALQ120', 'ALQ130', 'ALQ141', 'ALQ151', 'ALQpoly']]

## Dietary questionnaire - 82,091 (highly varied, will include all relevant for filtering later)

__from 99:__  
'DBQ100' - add salts  
'DBD270d' = helpings vegetables/day?  
__from 01__:  
'DBD102' = dark green vegetables /month    ...(9,000 responses)  
'DBD103' = cooked dried beans/peas  
__from 05, 07, 09, 11, 13:__  
'DBQ700' = how healthy is diet, generally  .........NANs: 2/5  
'DBD900' = # of meals from fast-food/pizza / week    .... NANs 1/4     
'DBD910' = # of frozen meals/pizzas / week .......NANs 1/10  
__all datasets:__  
x'DBQ197' = past 30 days milk consumption  
x'DBQ223' = type milk consumed  
'DBQpoly' = type milk * rate consumed......  
'DBQ235A/B/C' = milk over the ages  
'DBD895'. = # meals not home prepared/week  ......NANs: 1/10 


In [25]:
# import data from 1999-2013
dbq99 = pd.read_csv('CDCfiles/DBQ.csv')
dbq01 = pd.read_csv('CDCfiles/DBQ_B.csv')
dbq03 = pd.read_csv('CDCfiles/DBQ_C.csv')
dbq05 = pd.read_csv('CDCfiles/DBQ_D.csv')
dbq07 = pd.read_csv('CDCfiles/DBQ_E.csv')
dbq09 = pd.read_csv('CDCfiles/DBQ_F.csv')
dbq11 = pd.read_csv('CDCfiles/DBQ_G.csv')
dbq13 = pd.read_csv('CDCfiles/DBQ_H.csv')

In [26]:
######  RECODE 1999
dbq99['DBQ197'] = dbq99['DBD195'] 
dbq99['DBD895'] = dbq99['DBD090']
dbq99['DBD895'][dbq99['DBD895'] == 6666] = 0

dbq99['DBQ223A'] = dbq99['DBQ220A']
dbq99['DBQ223B'] = dbq99['DBQ220B']
dbq99['DBQ223C'] = dbq99['DBQ220C']
dbq99['DBQ223D'] = dbq99['DBQ220D']
dbq99['DBQ235A'] = dbq99['DBD235A']
dbq99['DBQ235B'] = dbq99['DBD235B']
dbq99['DBQ235C'] = dbq99['DBD235C']

##### RECODE 2001
dbq01['DBQ197'] = dbq01['DBD196']
dbq01['DBD895'] = dbq01['DBD090']
dbq01['DBD895'][dbq01['DBD895'] == 6666] = 0

dbq01['DBQ223A'] = dbq01['DBD221A']
dbq01['DBQ223B'] = dbq01['DBD221B']
dbq01['DBQ223C'] = dbq01['DBD221C']
dbq01['DBQ223D'] = dbq01['DBD221D']
dbq01['DBQ235A'] = dbq01['DBD235AE']
dbq01['DBQ235B'] = dbq01['DBD235BE']
dbq01['DBQ235C'] = dbq01['DBD235CE']

##### recode 2003
dbq03['DBQ197'] = dbq03['DBD197']
dbq03['DBD895'] = dbq03['DBD090']
dbq03['DBD895'][dbq03['DBD895'] == 6666] = 0

dbq03['DBQ223A'] = dbq03['DBQ221A']
dbq03['DBQ223B'] = dbq03['DBQ221B']
dbq03['DBQ223C'] = dbq03['DBQ221C']
dbq03['DBQ223D'] = dbq03['DBQ221D']

#### recode 2005
dbq05['DBD895'] = dbq05['DBD091']
dbq05['DBD895'][dbq05['DBD895'] == 6666] = 0

dbq05['DBQ223A'] = dbq05['DBD222A']
dbq05['DBQ223B'] = dbq05['DBD222B']
dbq05['DBQ223C'] = dbq05['DBD222C']
dbq05['DBQ223D'] = dbq05['DBD222D']

##### recode 2007
dbq07['DBQ223A'] = dbq07['DBD222A']
dbq07['DBQ223B'] = dbq07['DBD222B']
dbq07['DBQ223C'] = dbq07['DBD222C']
dbq07['DBQ223D'] = dbq07['DBD222D']

In [27]:
# combine data from 2003-2013
dbqnew = dbq03.append(dbq05).append(dbq07).append(dbq09).append(dbq11).append(dbq13)

In [28]:
# recode NaNs
dbqnew['DBQ700'][dbqnew['DBQ700'] > 6.0] = np.nan

# recode fast-food NaNs to 0 where meals out = 0.0
dbqnew['DBD900'][dbqnew['DBD895'] == 0.0] = 0.0
# recode >22 meals out / week to 22
dbqnew['DBD900'][dbqnew['DBD900'] == 5555] = 22
# recode NaNS
dbqnew['DBD900'][dbqnew['DBD900'] > 7750.0] = np.nan
dbqnew['DBD910'][dbqnew['DBD910'] > 7750.0] = np.nan

In [29]:
# combine 1999 and 2001 with post 2001
dbq = dbq99.append(dbq01).append(dbqnew)

# recode NaNs
dbq['DBQ197'][dbq['DBQ197'] > 6.0] = np.nan
dbq['DBQ197'][dbq['DBQ197'] == 4.0] = 2.0

# recoding milk fat: 5=whole, 3=reduced, 2= lowfat , 1=skim:
dbq['DBQ223'] = np.nan
dbq['DBQ223'][dbq['DBQ223A'] == 10.0] = 5.0
dbq['DBQ223'][dbq['DBQ223B'] == 11.0] = 3.0
dbq['DBQ223'][dbq['DBQ223C'] == 12.0] = 2.0
dbq['DBQ223'][dbq['DBQ223D'] == 13.0] = 1.0
dbq['DBQpoly'] = dbq['DBQ197']*dbq['DBQ223']

# recode > 22 meals out /week to 22, recode NaNs 
dbq['DBD895'][dbq['DBD895'] == 5555] = 22
dbq['DBD895'][dbq['DBD895'] > 7000] = np.nan

#recode NaNs
dbq['DBQ235A'][dbq['DBQ235A'] == 4.0] = 2.0
dbq['DBQ235A'][dbq['DBQ235A'] > 5.0] = np.nan
dbq['DBQ235B'][dbq['DBQ235B'] == 4.0] = 2.0
dbq['DBQ235B'][dbq['DBQ235B'] > 5.0] = np.nan 
dbq['DBQ235C'][dbq['DBQ235C'] == 4.0] = 2.0
dbq['DBQ235C'][dbq['DBQ235C'] > 5.0] = np.nan 

In [30]:
# pull out the just features we want
dbqF = dbq[['SEQN', 'DBD895', 'DBQ197', 'DBQ223', 'DBQpoly', 'DBQ235A', 'DBQ235B', 'DBQ235C', 'DBQ700', 'DBD900', 'DBD910']]

## Physical activity - 75,705 entries (varied)

'PAQ635' (/620) - bike/walk 10 minutes continously?  yes/no?   ...__NANs 25%__  
'PAQ640' (/050Q, scale U) - frequency of biking/walking  ...__NANs 25%__  
'PAD645' : # minutes per typical bike/walk  
'PAD645min' (/080) - avg # minutes walking/biking per trip  ...__NANs 25%__  
'PADvig1' : vigorous activity yes/no    
'PADvig2' : vigorous activity times/month   
'PADmod1' : moderate activity yes/no  
'PADmod2' : moderate activity times/month   
'PADtvcomp' : hours per day  
'PADtvpoly' : squared  

including both PAD645 (minutes biking walking/trip) AND PAD645min (multiplied by frequency (640)  
paqold.PAQMOD1 is taking YES from both house-tasks and moderate activity question (but only 50% overlap)
#### possible additions from old:
PAQ100 x PAD120 x PAD160 = (tasks around home/yard > light sweating or heartrate, NANs 30%)  
PAQ180 = avg level of physical activity each day (categorical) (NANs: 30%)  
Muscle Strengthening activities?  
PAQ560: times per week you play exercise hard?  (70% NANs)
PAQ540: compare with activity 10 yrs ago?

In [31]:
# import data form 1999 - 2013
paq99 = pd.read_csv('CDCfiles/PAQ.csv')
paq01 = pd.read_csv('CDCfiles/PAQ_B.csv')
paq03 = pd.read_csv('CDCfiles/PAQ_C.csv')
paq05 = pd.read_csv('CDCfiles/PAQ_D.csv')
paq07 = pd.read_csv('CDCfiles/PAQ_E.csv')
paq09 = pd.read_csv('CDCfiles/PAQ_F.csv')
paq11 = pd.read_csv('CDCfiles/PAQ_G.csv')
paq13 = pd.read_csv('CDCfiles/PAQ_H.csv')

In [32]:
# recode sedentary time into variable "PADTVcomp" 
paqolder = paq99.append(paq01)
paqolder['PADtvcomp'] = paqolder['PAQ480']
paqolder['PADtvcomp'][paqolder['PADtvcomp'] == 6.0] = 0.0
paqolder['PADtvcomp'][paqolder['PADtvcomp'] > 70.0] = np.nan   
paqolder['PADtvpoly'] = paqolder['PADtvcomp']*paqolder['PADtvcomp']

paqmid = paq03.append(paq05)
paqmid['PAD590'][paqmid['PAD590'] == 6.0] = 0.0
paqmid['PAD600'][paqmid['PAD600'] == 6.0] = 0.0
paqmid['PADtvcomp'] = paqmid['PAD590'] + paqmid['PAD600']
paqmid['PADtvcomp'][paqmid['PADtvcomp'] > 70.0] = np.nan   
paqmid['PADtvpoly'] = paqmid['PADtvcomp']*paqmid['PADtvcomp']

# combine 99 and 01 with 03 and 05
paqold = paqolder.append(paqmid)

In [33]:
# walked or bicycled last 30 days:
paqold['PAQ635'] = paqold['PAD020']
paqold['PAQ635'][paqold['PAQ635'] == 3.0] = 2.0
paqold['PAQ635'][paqold['PAQ635'] > 2.0] = np.nan
paqold['PAQ635'][paqold['PAQ635'] == 2.0] = 0.0

## of days walked/bicycled
paqold['PAQ640'] = paqold['PAQ050Q']
paqold['PAQ640'][paqold['PAQ640'] > 7500.0] = np.nan
paqold['PAQ640'][paqold['PAQ635'] == 0.0] = 0.0
paqold['PAQ640U'] = paqold['PAQ050U'].replace([1.0, 2.0, 3.0, np.nan], [365.0, 52.0, 12.0, 0.0])
paqold['PAQ640'] = paqold['PAQ640']*paqold['PAQ640U']

# minutes walked/bicycled
paqold['PAD645'] = paqold['PAD080']
paqold['PAD645'][paqold['PAD645'] > 7500.0] = 0
paqold['PAD645'][paqold['PAQ635'] == 0.0] = 0.0
paqold['PAD645min'] = paqold['PAD645']*paqold['PAQ640']

# vigorous activity, yes or no?
paqold['PADvig1'] = paqold['PAD200']  # yes/no in last 30 days
paqold['PADvig1'][paqold['PADvig1'] == 3.0] = 2.0
paqold['PADvig1'][paqold['PADvig1'] == 2.0] = 0.0
paqold['PADvig1'][paqold['PADvig1'] > 6.0] = np.nan

#  muscle strengthening, times/month?  (compare with vigorous activity in later years as vig2?)
paqold['vigmask99'] = paqold['PAD200']
paqold['vigmask99'][paqold['vigmask99'] == 2.0] = 0.0  # create mask for multiplying NANs back in at end
paqold['vigmask99'][paqold['vigmask99'] == 3.0] = 0.0

paqold['PADvig2'] = paqold['PAD460']*1.25  
    # scale these values to better match vig2 from later years
paqold['PADvig2'][paqold['PADvig2'].isnull()] = 0.0
paqold['PADvig2'] = paqold['PADvig2']*paqold['vigmask99']
    # so we code frequency as 0.0 for those who answered no in the yes/no section.  
    # then we restore the NaN's for those who didn't answer the yes/no question

#moderate activity, yes or no (using tasks around house(100) AND moderate activity(320))
paqold['PADmod1'] = paqold['PAQ100']
paqold['PADmod1'][paqold['PADmod1'] == 3.0] = 2.0
paqold['PADmod1'][paqold['PAD320'] == 1.0] = 1.0
paqold['PADmod1'][paqold['PADmod1'] == 2.0] = 0.0
paqold['PADmod1'][paqold['PADmod1'] > 6.0] = np.nan

# moderate activity, times/month
paqold['modmask99'] = paqold['PAQ100']
paqold['modmask99'][paqold['modmask99'] == 2.0] = 0.0
paqold['modmask99'][paqold['modmask99'] == 3.0] = 0.0
paqold['PADmod2'] = paqold['PAD120']
paqold['PADmod2'][paqold['PADmod2'].isnull()] = 0.0
paqold['PADmod2'] = paqold['PADmod2']*paqold['modmask99']

In [34]:
# sedentary time
paqmid2 = paq07.append(paq09)
paqmid2['PAD590'][paqmid2['PAD590'] == 6.0] = 0.0
paqmid2['PAD600'][paqmid2['PAD600'] == 6.0] = 0.0
paqmid2['PADtvcomp'] = paqmid2['PAD590'] + paqmid2['PAD600']
paqmid2['PADtvcomp'][paqmid2['PADtvcomp'] > 70.0] = np.nan   
paqmid2['PADtvpoly'] = paqmid2['PADtvcomp']*paqmid2['PADtvcomp']

paqnewer = paq11.append(paq13)
paqnewer['PAQ710'][paqnewer['PAQ710'] == 8.0] = 0.0
paqnewer['PAQ710'][paqnewer['PAQ710'] > 75.0] = np.nan
paqnewer['PAQ715'][paqnewer['PAQ715'] == 8.0] = 0.0
paqnewer['PAQ715'][paqnewer['PAQ715'] > 75.0] = np.nan
paqnewer['PADtvcomp'] = (paqnewer['PAQ710'] + paqnewer['PAQ715'])
paqnewer['PADtvpoly'] = paqnewer['PADtvcomp']*paqnewer['PADtvcomp']

In [35]:
# combine 07 and 09 with 11 and 13
paqnew = paqmid2.append(paqnewer)

#walk or bicycle yes/no
paqnew['PAQ635'][paqnew['PAQ635'] > 2.0] = np.nan
paqnew['PAQ635'][paqnew['PAQ635'] == 2.0] = 0.0

# frequency bike/walk
paqnew['PAQ640'][paqnew['PAQ640'] > 70.0] = np.nan
paqnew['PAQ640'][paqnew['PAQ635'] == 0.0] = 0.0
paqnew['PAQ640U'] = 52.0
paqnew['PAQ640'] = paqnew['PAQ640']*paqnew['PAQ640U']

# minutes spent biking/walking
paqnew['PAD645'][paqnew['PAD645'] > 7500.0] = np.nan
paqnew['PAD645'][paqnew['PAQ635'] == 0.0] = 0.0
paqnew['PAD645min'] = paqnew['PAD645']*paqnew['PAQ640']

# vigorous activity, YES/NO
paqnew['PADvig1'] = paqnew['PAQ650']  # vigorous rec (seems equivalent to 'last 30 days')
paqnew['PADvig1'][paqnew['PAQ605'] == 1] = 1  # combine with vigorous work
paqnew['PADvig1'][paqnew['PADvig1'] == 2] = 0.0
paqnew['PADvig1'][paqnew['PADvig1'] > 6.0] = np.nan

# vigorous activity, times/month
paqnew['vigmask11'] = paqnew['PAQ605']
paqnew['vigmask11'][paqnew['vigmask11'] == 2.0] = 0.0  # create mask for multiplying nans back in at end
paqnew['vigmask11'][paqnew['vigmask11'] > 6.0] = np.nan
paqnew['PADvig2'] = paqnew['PAQ655']*3.5  # of times per week (*3.5 to months, trying to normalize)vigorous rec
paqnew['PAQ610'][paqnew['PAQ610'].isnull()] = 0.0
paqnew['PADvig2'][paqnew['PADvig2'].isnull()] = 0.0
paqnew['PADvig2'] = paqnew['PADvig2']*paqnew['vigmask11']
paqnew['PADvig2'] = paqnew['PADvig2'] + paqnew['PAQ610']*3.5  # add in vigorous work
    # so we code frequency as 0.0 for those who answered no in the yes/no section.  
    # then we restore the NaN's for those who didn't answer the yes/no question
    
#moderate activity, yes or no
paqnew['PADmod1'] = paqnew['PAQ620']
paqnew['PADmod1'][paqnew['PAQ665'] == 1.0] = 1.0
paqnew['PADmod1'][paqnew['PADmod1'] > 6.0] = np.nan
paqnew['PADmod1'][paqnew['PADmod1'] == 2.0] = 0.0

# moderate activity, times/month
# NOTE THAT THIS MASK WORKS DIFFERENTLY THAN THE ABOVE
paqnew['PAQ620'][paqnew['PAQ620'] == 2.0] = 0.0
paqnew['PAQ665'][paqnew['PAQ665'] == 2.0] = 0.0
paqnew['modmask11'] = paqnew['PAQ620'] + paqnew['PAQ665']
paqnew['modmask11'][paqnew['modmask11'].isnull()] = 1.0
paqnew['modmask11'][paqnew['modmask11'] > 0.0] = 1.0
    # the mask holds 0's for all the subjects who answered NO to both work AND recreational 
    # moderate activity question.  the rest should stay nans (so all non 0's get multipled by 1).
paqnew['PAQ625'][paqnew['PAQ625'] > 75.0] = np.nan
paqnew['PAQ625'] = paqnew['PAQ625']*3.5
paqnew['PAQ625'][paqnew['PAQ625'].isnull()] = 999.0
paqnew['PAQ670'][paqnew['PAQ670'] > 75.0] = np.nan
paqnew['PAQ670'] = paqnew['PAQ670']*3.5
paqnew['PAQ670'][paqnew['PAQ670'].isnull()] = 999.0
paqnew['PADmod2'] = paqnew['PAQ670'] + paqnew['PAQ625']
paqnew['PADmod2'] = paqnew['PADmod2']*paqnew['modmask11']
paqnew['PADmod2'][paqnew['PADmod2'] == 1998.0] = np.nan
paqnew['PADmod2'][paqnew['PADmod2'] > 1000.0] = paqnew['PADmod2'] - 999.0

In [36]:
# combine data from before 2005 with later
paq = paqold.append(paqnew)

In [37]:
# pull out just the features we want
paqF = paq[['SEQN', 'PAQ635', 'PAQ640', 'PAD645', 'PAD645min', 'PADvig1', 'PADvig2', 'PADmod1', 'PADmod2', 'PADtvcomp', 'PADtvpoly']]

# Cardiovascular health, >40yrs only

#### 1999 has only:

'CDQ010' = shortness of breath on inclines

#### the rest also have:
'CDQ001': pain/discomfort in chest  
'CDQ002': ...when walking uphill  
'CDQ003': ...when walking ordinary pace  
'CDQ008': severe pain lasting more than 30 min?  
'CDQ009D': pain in upper sternum  

In [38]:
# import data from 1999 - 2013
cdq99 = pd.read_csv('CDCfiles/CDQ.csv')
cdq01 = pd.read_csv('CDCfiles/CDQ_B.csv')
cdq03 = pd.read_csv('CDCfiles/CDQ_C.csv')
cdq05 = pd.read_csv('CDCfiles/CDQ_D.csv')
cdq07 = pd.read_csv('CDCfiles/CDQ_E.csv')
cdq09 = pd.read_csv('CDCfiles/CDQ_F.csv')
cdq11 = pd.read_csv('CDCfiles/CDQ_G.csv')
cdq13 = pd.read_csv('CDCfiles/CDQ_H.csv')

In [39]:
# combine all data
cdq = cdq01.append(cdq03).append(cdq05).append(cdq07).append(cdq09).append(cdq11).append(cdq13)

In [40]:
#from 99 on, shortness of breath on inclines:
cdq['CDQ010'][cdq['CDQ010'] > 6] = np.nan
# from 01 on, ever had pain/discomfort in chest:
cdq['CDQ001'][cdq['CDQ001'] > 6] = np.nan
# from 01 on, get pain/discomfort in chest when walking uphill or hurrying
cdq['CDQ002'][cdq['CDQ002'] > 6] = np.nan
cdq['CDQ002'][cdq['CDQ001'] == 2] = 2
cdq['CDQ002'][cdq['CDQ002'] == 3] = 2
# from 01 on, get pain/discomfort in chest when walking ordinary pace
cdq['CDQ003'][cdq['CDQ003'] > 6] = np.nan
cdq['CDQ003'][cdq['CDQ001'] == 2] = 2
#from 01 on, severe chest pain lasting more than 30 min
cdq['CDQ008'][cdq['CDQ008'] > 2.0] = np.nan
cdq['CDQ008'][cdq['CDQ008'].isnull()] = 2.0
#from 01 on, pain in upper sternum:
cdq['CDQ009D'][cdq['CDQ009D'].isnull()] = 0.0

In [41]:
# pull out just the data we want
cdqF = cdq[['SEQN', 'CDQ010', 'CDQ001', 'CDQ002', 'CDQ003', 'CDQ008', 'CDQ009D']]

# Consumer Behavior

#### from 07 and 09, goldmine:
'CBQ020': fruits at home  
'CBQ030': green veggies at home  
'CBQ040': salty snacks   
'CBQ060': soft drinks   
'CBQ140': frequency of major shopping trips  
'CBD150': time to get to grocery store  
'CBD160': meals cooked at home per week

#### 07, 09, 11, 13:
'CBD070': money spent at grocery store  
'CBD090': money spent nonfood groceries  
'CBD110': money spent food other stores  
'CBD120': money spent eating out  
'CBD130': money spent carryout/delivered  

In [42]:
# import data from all years available (2007 thorugh 2013)
cbq07 = pd.read_csv('CDCfiles/CBQ_E.csv')
cbq09 = pd.read_csv('CDCfiles/CBQ_F.csv')
cbq11 = pd.read_csv('CDCfiles/CBQ_G.csv')
cbq13 = pd.read_csv('CDCfiles/CBQ_H.csv')

In [43]:
# combine 2007 and 2009
cbqold = cbq07.append(cbq09)

In [44]:
# recode NaNs
cbqold['CBQ020'][cbqold['CBQ020'] > 75.0] = np.nan
cbqold['CBQ030'][cbqold['CBQ030'] > 75.0] = np.nan
cbqold['CBQ040'][cbqold['CBQ040'] > 75.0] = np.nan
cbqold['CBQ060'][cbqold['CBQ060'] > 75.0] = np.nan

cbqold['CBQ140'][cbqold['CBQ140'] > 75.0] = np.nan
cbqold['CBD150'][cbqold['CBD150'] > 75000.0] = np.nan
cbqold['CBD160'][cbqold['CBD160'] > 750.0] = np.nan

In [45]:
# combine 2007 and 2009 with 2011 and 2013
cbq = cbqold.append(cbq11).append(cbq13)

In [46]:
# recode NaNs
cbq['CBD070'][cbq['CBD070'] > 750000.0] = np.nan
cbq['CBD090'][cbq['CBD090'] > 750000.0] = np.nan

cbq['CBD110'][cbq['CBD110'] > 750000.0] = np.nan
cbq['CBD120'][cbq['CBD120'] > 750000.0] = np.nan
cbq['CBD130'][cbq['CBD130'] > 750000.0] = np.nan

cbq['CBDfr1'] = cbq['CBD070']/cbq['CBD090']
cbq['CBDfr2'] = cbq['CBD070']/cbq['CBD120']
cbq['CBDfr3'] = cbq['CBD070']/cbq['CBD130']

cbq['CBDfr1'][cbq['CBDfr1'] == np.inf] = 20.0
cbq['CBDfr2'][cbq['CBDfr2'] == np.inf] = 20.0
cbq['CBDfr3'][cbq['CBDfr3'] == np.inf] = 20.0
cbq['CBDfr1'][cbq['CBDfr1'] > 20] = 20.0
cbq['CBDfr2'][cbq['CBDfr2'] > 20] = 20.0
cbq['CBDfr3'][cbq['CBDfr3'] > 20] = 20.0

In [47]:
# pull out just the features we want
cbqF = cbq[['SEQN', 'CBQ020', 'CBQ030', 'CBQ040', 'CBQ060', 'CBD070', 'CBD090', 'CBD110', 'CBD120', 'CBD130', 'CBQ140', 'CBD150', 'CBD160', 'CBDfr1', 'CBDfr2', 'CBDfr3']]

## Smoking - 51,149 entries

'SMQ020' : smoked 100 cigarettes in your life?  50/50, 0.1% NANs  
'SMQ030 : age started smoking?  (set non-smokers to 85)  
'SMQ040' : do you now smoke cigarettes?  
'SMD650' : avg #cigs in last 30 days    __MISSING FROM 99 and 01__

available all years?:
ADD: SMD050Q/U: how long since quit  
ADD: SMD057?  # smoked/day when quit?  
ADD: SMD070  # smoked/day now?  
ADD: SMD075: # years smoked this amount

available from 05 on:
SMD055: age when last smoked regularly?  


In [48]:
# import data from 1999 - 2013
smq99 = pd.read_csv('CDCfiles/SMQ.csv')
smq01 = pd.read_csv('CDCfiles/SMQ_B.csv')
smq03 = pd.read_csv('CDCfiles/SMQ_C.csv')
smq05 = pd.read_csv('CDCfiles/SMQ_D.csv')
smq07 = pd.read_csv('CDCfiles/SMQ_E.csv')
smq09 = pd.read_csv('CDCfiles/SMQ_F.csv')
smq11 = pd.read_csv('CDCfiles/SMQ_G.csv')
smq13 = pd.read_csv('CDCfiles/SMQ_H.csv')

In [49]:
# combine all years
smq = smq99.append(smq01).append(smq03).append(smq05).append(smq07).append(smq09).append(smq11).append(smq13)

In [50]:
# pull in age from demo
smq = smq.set_index('SEQN').join(demo[['SEQN', 'RIDAGEYR']].set_index('SEQN')) 
smq['SEQN'] = smq.index

In [51]:
import warnings; warnings.filterwarnings('ignore')
smq['SMQ020'][smq['SMQ020'] > 2.0] = np.nan
smq['SMQ020'][smq['SMQ020'] == 2.0] = 0.0
    ####  SMD020 can now serve as a mask!  set isnull() = 0.0 in target, then multiply w mask 
    ####  to put 0.0's for subjects who never really smoked, 1.0's to maintain those that did, and
    ####  np.nan for the initial missing values

# # per day when quit?
smq['SMD057'][smq['SMD057'].isnull()] = 0.0
smq['SMD057'] = smq['SMD057']*smq['SMQ020'] 
smq['SMD057'][smq['SMD057'] > 750.0] = np.nan

# age when last smoked regularly
smq['SMD055'][smq['SMD055'].isnull()] = 0.0
smq['SMD055'][smq['SMD055'] > 750] = np.nan
#age started smoking
smq['SMD030'][smq['SMD030'].isnull()] = 0.0
smq['SMD030'][smq['SMD030'] > 750] = np.nan
# apply mask to keep those 0's that should be 0's 0's
smq['SMD055'] = smq['SMD055']*smq['SMQ020'] 
smq['SMQ040'] = smq['SMQ040']*smq['SMQ020'] 

# years smoked regularly
smq['SMDlen'] = smq['SMD055'] - smq['SMD030']
smq['SMDlen'][smq['SMDlen'] < 0.0] = smq['RIDAGEYR'] - smq['SMD030']

# intake x years
smq['SMDamt'] = smq['SMDlen']*smq['SMD057']
smq['SMDamt'][smq['SMDamt'] == 0.0] = smq['SMDlen']*smq['SMD650']
smq['SMDamt'][smq['SMDamt'].isnull()] = 0.0
smq['SMDamt'] = smq['SMDamt']*smq['SMQ020'] 

# fix age started smoking...
smq['SMD030'][smq['SMQ020'] == 0.0] = 85.0
smq['SMD030'][smq['SMD030'] > 700.0] = 85.0
smq['SMD030'][smq['SMD030'].isnull()] = 85.0
smq['SMD030'][smq['SMQ020'].isnull()] = np.nan
    # this is a weird one, we'll try setting the age = 80.0 for all non-responderes 
    ###  WE COULD CHANGE THIS, FILL IN MEAN STARTING AGE FOR THE ~2,000 MISSING VALUES FROM 020
    
#smq['SMQ040'][smq['SMQ040'] == 3.0] = 0.0
#smq['SMQ040'][smq['SMQ040'] == 1.0] = 3.0
smq['SMQ040'][smq['SMQ040'] > 6.0] = np.nan
smq['SMQ040'][smq['SMQ040'].isnull()] = 0.0
smq['SMQ040'] = smq['SMQ040']*smq['SMQ020'] 

smq['SMD650'][smq['SMD650'] == 777.0] = np.nan
smq['SMD650'][smq['SMD650'] == 999.0] = np.nan
smq['SMD650'][smq['SMD650'].isnull()] = 0.0
smq['SMD650'] = smq['SMD650']*smq['SMQ020']

In [52]:
# pull out just the features we want
smqF = smq[['SEQN', 'SMQ020', 'SMD030','SMQ040', 'SMD650', 'SMD057', 'SMDlen', 'SMDamt']]

# Income

#### Available for 07, 09, 11, 13

'IND235': monthly family income (categorical 1-12)  .......NANs: 1/10  
'INDFMMPI': monthly poverty index (ordinal 0 to 5)  .......NANs: 1/10  
'INDFMMPC': monthly poverty index (categorical 1,2,3)  .......NANs: 1/20?  

In [53]:
# import data from all years available
inc07 = pd.read_csv('CDCfiles/INQ_E.csv')
inc09 = pd.read_csv('CDCfiles/INQ_F.csv')
inc11 = pd.read_csv('CDCfiles/INQ_G.csv')
inc13 = pd.read_csv('CDCfiles/INQ_H.csv')
# combine 2007 through 2013
inc = inc07.append(inc09).append(inc11).append(inc13)
# recode NaNs
inc['IND235'][inc['IND235'] > 75.0] = np.nan
inc['INDFMMPC'][inc['INDFMMPC'] > 6.0] = np.nan
# pull out the features of interest
incF = inc[['SEQN', 'IND235', 'INDFMMPI', 'INDFMMPC']]

# Current medical conditions

'MCQ160A': ever told you have arthritis?   ... ... NANs: 1/2, 25% YES  
'MCQ160I' (/160M) : ever told you have thyroid disease?   .... NANs: 1/2, 7% YES  
...(also: ever told you had heart attack, stroke, emphysema, bronchitis, coronary heart disease)  

'MCQ053': taking treatment for anemia?   ... ... full response, 3% positive
'MCQ250A' (/300C) : blood relatives have diabetes?     ... ... NANs: 1/2  
'MCQ250D': blood relatives have arthritis?     ... ... NANs: 1/2  
'MCQ250E': (in osteoporosis section later) blood relatives have osteopororis?     ... ... NANs: 1/2  
'MCQ250F': blood relatives have hypertension/stroke?     ... ... NANs: 1/2  

In [54]:
# import data from all years available
mcq99 = pd.read_csv('CDCfiles/MCQ.csv')
mcq01 = pd.read_csv('CDCfiles/MCQ_B.csv')
mcq03 = pd.read_csv('CDCfiles/MCQ_C.csv')
mcq05 = pd.read_csv('CDCfiles/MCQ_D.csv')
mcq07 = pd.read_csv('CDCfiles/MCQ_E.csv')
mcq09 = pd.read_csv('CDCfiles/MCQ_F.csv')
mcq11 = pd.read_csv('CDCfiles/MCQ_G.csv')
mcq13 = pd.read_csv('CDCfiles/MCQ_H.csv')
osq05 = pd.read_csv('CDCfiles/OSQ_D.csv')
osq07 = pd.read_csv('CDCfiles/OSQ_E.csv')
osq09 = pd.read_csv('CDCfiles/OSQ_F.csv')
osq13 = pd.read_csv('CDCfiles/OSQ_H.csv')

In [55]:
# recoding questions that changed in later years
mcq99['MCQ300A'] = mcq99['MCQ250F']
mcq01['MCQ300A'] = mcq01['MCQ250F']
mcq03['MCQ300A'] = mcq03['MCQ250F']

mcq99['OSQ150'] = mcq99['MCQ250E']
mcq01['OSQ150'] = mcq01['MCQ250E']
mcq03['OSQ150'] = mcq03['MCQ250E']

mcq99['MCQ300C'] = mcq99['MCQ250A']
mcq01['MCQ300C'] = mcq01['MCQ250A']
mcq03['MCQ300C'] = mcq03['MCQ250A']

mcq05['OSQ150'] = osq05['OSQ150']
mcq07['OSQ150'] = osq07['OSQ150']
mcq09['OSQ150'] = osq09['OSQ150']
mcq13['OSQ150'] = osq13['OSQ150']

In [56]:
# combine all years
mcq = mcq99.append(mcq01).append(mcq03).append(mcq05).append(mcq07).append(mcq09).append(mcq11).append(mcq13)

In [57]:
# filter NaNs
mcq['MCQ300C'][mcq['MCQ300C'] > 5.0] = np.nan
mcq['MCQ300C'][mcq['MCQ300C'] == 2.0] = 0

mcq['MCQ300A'][mcq['MCQ300A'] > 5.0] = np.nan
mcq['MCQ300A'][mcq['MCQ300A'] == 2.0] = 0

mcq['OSQ150'][mcq['OSQ150'] > 5.0] = np.nan
mcq['OSQ150'][mcq['OSQ150'] == 2.0] = 0.0

In [58]:
# pull out the features we want
mcqF = mcq[['SEQN', 'MCQ300A', 'OSQ150', 'MCQ300C']]

# Current medications
#### Some patients are already being treated for various conditions.  Here we extract records, below we will combine them with 'at risk' measures

In [59]:
# import data from all years available
bpq99 = pd.read_csv('CDCfiles/BPQ.csv')
bpq01 = pd.read_csv('CDCfiles/BPQ_B.csv')
bpq03 = pd.read_csv('CDCfiles/BPQ_C.csv')
bpq05 = pd.read_csv('CDCfiles/BPQ_D.csv')
bpq07 = pd.read_csv('CDCfiles/BPQ_E.csv')
bpq09 = pd.read_csv('CDCfiles/BPQ_F.csv')
bpq11 = pd.read_csv('CDCfiles/BPQ_G.csv')
bpq13 = pd.read_csv('CDCfiles/BPQ_H.csv')
diq99 = pd.read_csv('CDCfiles/DIQ.csv')
diq01 = pd.read_csv('CDCfiles/DIQ_B.csv')
diq03 = pd.read_csv('CDCfiles/DIQ_C.csv')
diq05 = pd.read_csv('CDCfiles/DIQ_D.csv')
diq07 = pd.read_csv('CDCfiles/DIQ_E.csv')
diq09 = pd.read_csv('CDCfiles/DIQ_F.csv')
diq11 = pd.read_csv('CDCfiles/DIQ_G.csv')
diq13 = pd.read_csv('CDCfiles/DIQ_H.csv')

In [60]:
# combine data from all years
bpq = bpq99.append(bpq01).append(bpq03).append(bpq05).append(bpq07).append(bpq09).append(bpq11).append(bpq13)
diq = diq99.append(diq01).append(diq03).append(diq05).append(diq07).append(diq09).append(diq11).append(diq13)

In [61]:
# recode NaNs to 'not treated' in this case.  a few of these may be wrong, but its better not to 
#lose thousands of samples here just because we don't know 100% certain they aren't on meds
bpq.BPQ040A[bpq.BPQ040A > 1.0] = 0.0
bpq.BPQ050A[bpq.BPQ050A > 1.0] = 0.0
bpq.BPQ080[bpq.BPQ080 > 1.0] = 0.0
bpq.BPQ100D[bpq.BPQ100D > 1.0] = 0.0
diq.DIQ010[diq.DIQ010 > 1.0] = 0.0

In [62]:
# pull out the features we want
bpbpF = bpq[['SEQN', 'BPQ050A']]  #try BPQ040A as alternative
bpcholF = bpq[['SEQN', 'BPQ100D']]  #try BPQ080 as alternative
diqF = diq[['SEQN', 'DIQ010']]

## Save cleaned data into .csv for subsequent modeling...

In [ ]:
# cholesterol targeting can be adjusted here (predict based off of total, LDL - HDL, or triglycerides...)
# with cholesterol, we can try modeling total cholesterol, or LDL - HDL,
# here we set target as total cholesterol
target = 'LBXTC' 
cholF['combo'] = (cholF['LBDLDL'] - cholF['LBDHDD'])
totalcholF = cholF[['SEQN', target]]
          
# join together all features and targets into the starting dataframe 'XY'    
XY = demoFdummy.set_index('SEQN').join(bodyF.set_index('SEQN')).join(alcF.set_index('SEQN')).join(dbqF.set_index('SEQN')).join(paqF.set_index('SEQN')).join(smqF.set_index('SEQN')).join(cbqF.set_index('SEQN')).join(mcqF.set_index('SEQN')).join(bpxF.set_index('SEQN')).join(bpbpF.set_index('SEQN')).join(totalcholF.set_index('SEQN')).join(bpcholF.set_index('SEQN')).join(ghbF.set_index('SEQN')).join(diqF.set_index('SEQN'))

In [ ]:
XY.to_csv('NHANES_XY.csv')